In [2]:
## Importing libraries
import PIL
import imageio as imageio
import pandas as pd
import numpy as np
import cv2
import os
import glob
import time
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import tensorflow as tf
import seaborn as sns

from tensorflow.python.keras import Sequential, models
from tensorflow.python.keras.layers import Flatten, Dense, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
import keras
import matplotlib.pyplot as plt

In [3]:
###################################################################################

# draw an image with detected objects
def draw_image_with_boxes(filename, result_list):
    # load the image
    data = pyplot.imread(filename)
    # plot the image
    pyplot.imshow(data)
    # get the context for drawing boxes
    ax = pyplot.gca()
    # plot each box
    for result in result_list:
        # get coordinates
        x, y, width, height = result['box']
        # create the shape
        rect = Rectangle((x, y), width, height, fill=False, color='red')
        # draw the box
        ax.add_patch(rect)
    # show the plot
    pyplot.show()




#################################################################################3

In [5]:




path = "C:/Users/krzys/Desktop/Polibuda/sem. 6/BIAI/przetrenowane"

dataset = {
    "image_path": [],
    "mask_status": [],
    "where": []
}

for where in os.listdir(path):
    for status in os.listdir(path + "/" + where):
        for image in glob.glob(path + where + "/" + status + "/" + "*.png"):
            dataset["image_path"].append(image)
            dataset["mask_status"].append(status)
            dataset["where"].append(where)

dataset = pd.DataFrame(dataset)
dataset.head()

,image_path,mask_status,where


In [ ]:
## Konwersja na numpy (potrzebne dp TensorFlowa)
X = np.array(X)
y = np.array(y)

## Normalizacja danych
X = X/255

## Inicjalizacja procesu trenowania, okreslenie funkcji aktywnacji

model = Sequential() # w keras są 2 modele - sekwencyjny i funkcjonalny. Sekwencyjny pozwala buydować model warstwa po warstiw
model.add(Conv2D(64, (3, 3), activation = "relu")) # i tu są te warstwy po kolei dodawane - tu np warstwa konwolucyjna. Konwolucja = branie danych i filtrowanie, przez co algorytm się uczy
model.add(MaxPool2D(pool_size=(3, 3))) #maxpooling służy do przyspieszania, bo upraszcza obliczenia bez utraty istotnych informacji
model.add(Flatten()) # spłaszcza dane do postaci jednego wektora, by mogło wejść do wartwy dense
model.add(Dense(1, activation = "sigmoid"))

## kompilacja
model.compile(optimizer='adam', # typ optymalizatora , adam zazwyczaj najlepszy
              loss=tf.keras.losses.BinaryCrossentropy(), # funkcja straty, ewaluuje jak dobrze się algorytm się sprawuje, tą funkcje się minimalizuje
              metrics=['accuracy']) # miary mierzenia skuteczności algorytmu

## Reshaping training set to match Conc2D
X_train = X_train.reshape(len(X_train), X_train.shape[1], X_train.shape[2], 1)
X_val = X_val.reshape(len(X_val), X_val.shape[1], X_val.shape[2], 1)

history = model.fit(X_train, y_train, epochs=5, batch_size = 32) # trenowańsko  jedna epoka = 1 przejście przez cały zbiór treningowy, 1 akutualizacja wag, i 1 przejście z powrtotem. Potem ładuje nowe dane, dla kolejnej epoki
                                                                 # dane podaje się w batchach = wierszach, podaje je algorytmowi w takich mniejszych porcjach, przez to się efektywniej uczy
model.summary() # wyświetla podsumowanie

model.evaluate(X_val, y_val) # ewaluacja skuteczności przez model walidacji i na tej podstawie liczy jak bardzo dobry jest ten model

## Wykonanie predykcji
prediction = model.predict_classes(X_val) # predykcja tego do której klasy ma iść dany obrazek - czy z maską, czy do bez maski

print("Raport zbiorowy")
print(classification_report(y_val, prediction))
print(confusion_matrix(y_val, prediction))

# dla predykcji zwyklego jednego obrazka
#print("Raport indywidualny")


#img_array = cv2.imread(path_to_image)

#model.save("model")
#img = img.reshape(len(X_val), X_val.shape[1], X_val.shape[2], 1)
#individual_prediction = model.predict(img)

#print(classification_report(individual_prediction))